In [1]:
import cv2
import json
import os
import pandas as pd
from math import sqrt,atan2,floor

In [2]:
def HOG(Img, no_divs=3, no_bins=6):

    n_HOG = no_divs*no_divs*no_bins;
    pi = 3.14
    #Initialize output HOG vector
    HOG = [0.0]*n_HOG

    #Apply sobel on image to find x and y orientations of the image
    Icv = Img#.getNumpyCv2()

    Ix = cv2.Sobel(Icv, ddepth = cv2.CV_32F, dx=1, dy=0, ksize=3)
    Iy = cv2.Sobel(Icv, ddepth = cv2.CV_32F, dx=0, dy=1, ksize=3)
    # print (len(Ix),len(Ix[0]))
    Ix = Ix.transpose()#(1,0,2)
    Iy = Iy.transpose()#(1,0,2)
    height,width = Img.shape
    cellx = width/no_divs     #width of each cell(division)
    celly = height/no_divs    #height of each cell(division)

    #Area of image
    img_area = height * width

    #Range of each bin
    BIN_RANGE = (2*pi)/(no_bins)

    m=0
    # print (Img.size)
    # print (Ix.shape)

    while m < no_divs:
        n = 0
        while n < no_divs:
            i = 0
            while i < cellx:
                j=0
                while j < celly:
                    # print (m*cellx+i)
                    # print (n*celly+j)
                    px = Ix[int(m*cellx +i), int(n*celly+j)]
                    py = Iy[int(m*cellx+i), int(n*celly+j)]

                    #grad value
                    grad = sqrt(px*px + py*py)
                    #normalized grad value
                    norm_grad = grad/img_area
                    #print norm_grad
                    #Angle
                    angle = atan2(py,px)
                    if(angle < 0):
                        angle = angle+ 2*pi
                    nth_bin = floor(float(angle/BIN_RANGE))
                    #nth_bin = angle*(180/pi)%180
                    #print nth_bin
                    HOG[((m*no_divs+n)*no_bins + int(nth_bin))] += norm_grad
                    j=j+1
                i= i+1
            n=n+1
        m=m+1

    return HOG

In [3]:
responses=[]
features=[]
print('Reading configurations')
conf = json.load(open('conf.json', 'r'))
database = conf['database']
savepath = conf['database_save_path']

Reading configurations


In [4]:
alist_filter = ['png','jpg','bmp','gif']
for dirName, subdirList, fileList in (os.walk(database)):
    file_path = os.path.join(dirName,'utf8')
    fp=open(str(file_path),"r",encoding='utf8')
    label_u=(fp.readline().strip())
    print (label_u ,len(fileList))

    for fname in fileList:
        if fname[-3:] in alist_filter:
            responses.append(label_u)
            file_path = os.path.join(dirName, fname)
            img=cv2.imread(str(file_path),0)
            im = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,243,50)
            
            features.append(HOG(im))
    

ർ 2
ണ്ണ 69
' 395
പ 522
3 149
ദ്ധ 111
ാ 575
ജ്ജ 8
1 133
അ 526
, 247
ൂ 276
ൾ 159
ഹ്മ 5
ന 539
ന്ദ 112
ദ 436
ഷ 397
ശ 472
) 219
ണ്ഡ 33
ല 533
4 112
്വ 88
ൾ 314
ഹ്ന 4
ഗ്ന 8
റ 526
5 111
7 226
മ 584
ഹ 382
0 190
ന്മ 58
ട 511
ഡ 204
ൺ 78
ഫ 169
6 144
ച്ച 391
ക്ത 79
ക്ഷ 225
+ 15
ള്ള 301
ച്ഛ 16
ഋ 54
റ്റ2 21
2 152
ഗ്ല 11
ക 685
യ 632
. 192
ഒ 436
യ്യ 77
ത്ന 8
ഉ 273
െ 579
ആ 405
ട്ട 475
ൽ 523
ൃ 137
ന്ത 260
റ്റ 212
ണ 558
ഗ 389
ഝ 48
ത്മ 18
ഖ 164
- 76
ഴ 347
ണ്മ 13
9 228
ന്റ 225
ഠ 474
ർ 539
ഞ 176
ങ്ക 159
മ്പ 141
? 478
സ 564
ബ 352
ഗ്ഗ 10
( 240
' 152
ച 441
ങ 46
ഏ 143
ജ്ഞ 12
് 593
ത്സ 50
ു 545
ി 563
സ്ഥ 67
ന്ഥ 10
വ്വ 9
ബ്ബ 6
ത്ഭ 11
മ്മ 123
്ര 246
മ്ല 6
ീ 335
ഢ 44
ഇ 492
ശ്ച 16
ക്ക 591
ത്ഥ 60
ധ 351
ഞ്ച 57
എ 527
ദ്ദ 129
ം 212
" 5
പ്പ 411
ല്ല 326
ണ്ട 429
േ 555
ഭ 359
ന്ധ 103
ത 577
ങ്ങ 442
ത്ത 543
്യ 399
ള 592
ര 578
ൗ 254
ജ 354
വ 574
8 200
ൻ 419
ഛ 47
ന്ന 526
ഥ 150
ഞ്ഞ 228
ഗ്മ 8
ഘ 134


In [6]:
df = pd.DataFrame(features)
df['target'] = responses

print('Saving to file')
#df = df.iloc[np.random.permutation(len(df))]
df.to_pickle(savepath)

Saving to file
